In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.seasonal import seasonal_decompose

In [18]:
imb_df = pd.read_csv('../data/imbalance_de.csv', squeeze=True, parse_dates=True, header=None)
imb_df.head()

,0,1,2
0,2013-12-31 23:00:00,707.500,156.36
1,2013-12-31 23:15:00,966.296,81.23
2,2013-12-31 23:30:00,1018.984,104.01
3,2013-12-31 23:45:00,867.940,60.51
4,2014-01-01 00:00:00,1738.004,239.03


In [19]:
imb_df.shape

(257010, 3)

In [22]:
imb_df['datetime'] = pd.to_datetime(imb_df[0])
imb_df.drop([0], axis=1, inplace=True)
imb_df.head()

,1,2,datetime
0,707.500,156.36,2013-12-31 23:00:00
1,966.296,81.23,2013-12-31 23:15:00
2,1018.984,104.01,2013-12-31 23:30:00
3,867.940,60.51,2013-12-31 23:45:00
4,1738.004,239.03,2014-01-01 00:00:00


In [23]:
imb_df.isnull().sum()

1           203
2             0
datetime      0
dtype: int64

In [24]:
imb_df.isna().sum()

1           203
2             0
datetime      0
dtype: int64

In [40]:
imb_df_groupby[['datetime']]

In [38]:
imb_df_groupby['hour'] = imb_df_groupby['datetime'].dt.hour
imb_df_groupby.head()

AttributeError: 'SeriesGroupBy' object has no attribute 'dt'

In [ ]:
speedy_groupby[‘dt_start_utc’].dt.to_period(‘D’)

In [36]:
imb_df_groupby = imb_df.groupby(by='datetime')
imb_df_groupby.head()

,1,2,datetime
0,707.500,156.36,2013-12-31 23:00:00
1,966.296,81.23,2013-12-31 23:15:00
2,1018.984,104.01,2013-12-31 23:30:00
3,867.940,60.51,2013-12-31 23:45:00
4,1738.004,239.03,2014-01-01 00:00:00
...,...,...,...
257005,82.501,40.96,2021-05-20 13:00:00
257006,-11.440,25.53,2021-05-20 14:30:00
257007,-346.008,18.48,2021-05-21 20:15:00
257008,30.308,111.81,2021-05-22 20:15:00


OK, ich bin nicht so weit gekommen. Aber die Stunden und Tage müssen vor dem groupby extrahiert werden, das geht mit dem groupby-object nicht. Danach sollte es einfach gehen inkl. Aggregation auf den Median. 